In [3]:
import torch
import matplotlib.pyplot as plt
from torch import optim, distributions, nn
from tqdm.autonotebook import tqdm
from sklearn.datasets import fetch_olivetti_faces
from gpzoo.kernels import NSF_RBF
from gpzoo.gp import NSF